<div class="alert alert-info">
    
➡️ Make sure that you have read the **[rules for hand-in assignments](https://www.ida.liu.se/~TDDE16/exam.en.shtml#handins)** and the **[policy on cheating and plagiarism](https://www.ida.liu.se/~TDDE16/exam.en.shtml#cheating)** before starting with this lab.

➡️ Make sure you fill in any cells (and _only_ those cells) that say **`YOUR CODE HERE`** or **YOUR ANSWER HERE**, and do _not_ modify any of the other cells.

➡️ **Before you submit your lab, make sure everything runs as expected.** For this, _restart the kernel_ and _run all cells_ from top to bottom. In Jupyter Notebook version 7 or higher, you can do this via "Run$\rightarrow$Restart Kernel and Run All Cells..." in the menu (or the "⏩" button in the toolbar).

</div>

# L1: Information Retrieval

In this lab you will apply basic techniques from information retrieval to implement the core of a minimalistic search engine. The data for this lab consists of a collection of app descriptions scraped from the [Google Play Store](https://play.google.com/store/apps?hl=en). From this collection, your search engine should retrieve those apps whose descriptions best match a given query under the vector space model.

In [1]:
# Define some helper functions that are used in this notebook

from IPython.display import display, HTML

def success():
    display(HTML('<div class="alert alert-success"><strong>Solution appears correct!</strong></div>'))

## Data set

The app descriptions come in the form of a compressed [JSON](https://en.wikipedia.org/wiki/JSON) file. Start by loading this file into a [Pandas](https://pandas.pydata.org) [DataFrame](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html#dataframe).

In [2]:
import bz2
import numpy as np
import pandas as pd

with bz2.open('app-descriptions.json.bz2') as source:
    df = pd.read_json(source)

In Pandas, a DataFrame is a table with indexed rows and labelled columns of potentially different types. You can access data in a DataFrame in various ways, including by row and column. To give an example, the code in the next cell shows rows 200–204:

In [3]:
df

,name,description
0,10000000,﻿10000000 is a Dungeon Crawling Puzzle RPG Mat...
1,1177 Vårdguiden,I 1177 Vårdguidens app får du tillgång till 11...
2,123 Draw🎨 Toddler counting for kids Drawing games,Need counting games for kids & drawing for tod...
3,123 Kids Fun MUSIC - Kids Music Educational Games,Beautiful and simple music application for tod...
4,123 number games for kids - Count & Tracing,A Fun and intuitive numbers game for your baby...
...,...,...
1609,고스톱 PLUS (무료 맞고 게임),심심할 때 한번씩 즐기는 고스톱(맞고).\n복잡하고 요란한 화면이 부담스러우셨나요?...
1610,기적의 검,"소지섭, 안젤리나가 선택한 바로 그 게임!\n\n★ 공식카페에서 구글 사전예약 한정..."
1611,이사만루2019,[2019년 8월 업데이트]\n- 신규 레전드 카드 추가\n- 메인 화면 이벤트 시...
1612,컴투스프로야구2019,★ 레전드 등급 추가로 더욱 강력해진 2019 메이저 업데이트 ★\n\n# 대한민국...


As you can see, there are two labelled columns: `name` (the name of the app) and `description` (a textual description). The code in the next cell shows how to access fields from the description column.

In [4]:
df['description'][200:205]

200    Introducing the best Brick Breaker game that e...
201    Classic Brick Game!\n\nBrick Classic is a popu...
202    Bricks Breaker - Glow Balls is a addictive and...
203    How to play\n- The ball flies to wherever you ...
204    Fight brave soldiers from around the globe on ...
Name: description, dtype: object

## Problem 1: Preprocessing

Your first task is to implement a preprocessor for your search engine. In the vector space model, *preprocessing* refers to any transformation applied to a text before vectorisation. Here you can restrict yourself to a simple type of preprocessing: tokenisation, stop word removal, and lemmatisation.

To implement your preprocessor, you can use [spaCy](https://spacy.io). Make sure to read the [Linguistic annotations](https://spacy.io/usage/spacy-101#annotations) section of the spaCy&nbsp;101; that section contains all the information you need for this problem (and more).

Implement your preprocessor by completing the skeleton code in the next cell, adding additional code as you deem necessary.

In [5]:
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])

def preprocess(text):                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
    """Preprocess the given text by tokenising it, removing any stop words, 
    replacing each remaining token with its lemma (base form), and discarding 
    all lemmas that contain non-alphabetical characters.

    Arguments:
      text (str): The text to preprocess.

    Returns:
      The list of remaining lemmas after preprocessing (represented as strings).
    """
    doc = nlp(text) # Transform into spaCy object.
    
    return [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

### 🤞 Test your code

Test your implementation by running the following cell:

In [6]:
"""Check that the preprocessing returns the correct output for a number of test cases."""

assert (
    preprocess('Apple is looking at buying U.K. startup for $1 billion') ==
    ['Apple', 'look', 'buy', 'startup', 'billion']
)
assert (
    preprocess('"Love Story" is a country pop song written and sung by Taylor Swift.') ==
    ['Love', 'Story', 'country', 'pop', 'song', 'write', 'sing', 'Taylor', 'Swift']
)
success()

## Problem 2: Vectorising

Your next task is to vectorise the data – and more specifically, to map each app description to a tf–idf vector. For this you can use the [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) class from [scikit-learn](https://scikit-learn.org/stable/). Make sure to specify your preprocessor from the previous problem as the `tokenizer` &ndash; not the `preprocessor`! &ndash; for the vectoriser. (In scikit-learn terminology, the `preprocessor` handles string-level preprocessing.)

After running the following cell:
- `vectorizer` should contain the vectorizer fitted on `df['description']`
- `X` should contain the vectorized `df['description']`

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(tokenizer = preprocess)
X = vectorizer.fit_transform(df['description'])

/Users/jakobberggren/miniforge3/envs/liu-text-mining/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


# 🤞 Test your code

Test your implementation by running the following cell:

In [8]:
"""Check that the dimensions of X are as expected."""

print(f"The dimensions of X are: {X.shape}")
assert X.shape[0] == 1614
assert 21200 < X.shape[1] < 21500

success()

The dimensions of X are: (1614, 21357)


The dimensions of `X` should be around 1614$\times$21356; the number of rows should be _exactly_ 1614 , while the number of columns may differ from that given here depending on the version of spaCy and the version of the language model used, as well as the pre-processing.

## Problem 3: Retrieving

To complete the search engine, your last task is to write a function that returns the most relevant app descriptions for a given query. An easy way to solve this task is to use scikit-learn&rsquo;s [NearestNeighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html) class. That class implements unsupervised nearest neighbours learning and allows you to easily find a predefined number of app descriptions whose vector representations are closest to the query vector.

First, instantiate and fit a class that returns the _ten (10)_ nearest neighbors:

In [9]:
"""Instantiate and fit a class that returns the 10 nearest neighbors."""

from sklearn.neighbors import NearestNeighbors

neigh = NearestNeighbors(n_neighbors=10)
neigh.fit(X)

NearestNeighbors(n_neighbors=10)

Second, implement a function that uses the fitted class to find the nearest neighbours for a given query string:

In [10]:
def search(query):
    """Find the nearest neighbors in `df` for a query string.

    Arguments:
      query (str): A query string.

    Returns:
      The 10 apps (with name and description) most similar (in terms of
      cosine similarity) to the given query as a Pandas DataFrame.
    """    
    #transform query into vector
    vec_query = vectorizer.transform([query])
    #extract the indicies of the 10 nearest vectors, return_distance=False to not include another column with euclidean distance
    index_list = neigh.kneighbors(vec_query, return_distance=False)
    #map all the values in index_list to the corresponding entry in df['name'], transpose to get the right dims
    return pd.DataFrame([df['name'].iloc[i] for i in index_list]).T

### 🤞 Test your code

Test your implementation by running the following cell, which will show the 10 best search results for the query _"dodge trains"_:

In [11]:
"""Check that searching for "dodge trains" returns a DataFrame with ten results,
   and that the top result is "Subway Surfers"."""

result = search('dodge trains')
display(result)
assert isinstance(result, pd.DataFrame), "Search results should be a Pandas DataFrame"
assert len(result) == 10, "Should return 10 search results"
assert result.iloc[0]["name"] == "Subway Surfers", "Top search result should be 'Subway Surfers'"
success()

,name
1301,Subway Surfers
1428,Train Conductor World
1394,Tiny Rails
1300,Subway Princess Runner
998,No Humanity - The Hardest Game
1429,Train for Animals - BabyMagica free
1168,Rush
786,LEGO® DUPLO® Train
1465,Virus War - Space Shooting Game
192,Boxing Star


The top hit in the list should be _Subway Surfers_.

## Problem 4: Finding terms with low/high idf

Recall that the inverse document frequency (idf) of a term is the lower, the more documents from a given collection the term appears in. To get a better understanding for this concept, your next task is to write code to find out which terms from the app descriptions have the lowest/highest idf.

Start by sorting the terms in _increasing_ order of idf, breaking ties by falling back on alphabetic order, and store the result in the variable `terms`.

In [12]:
feature_names = vectorizer.get_feature_names_out()
idf_values = vectorizer.idf_ # idf-values

data = [(feature_name, idf) for feature_name, idf in zip(feature_names, idf_values)]

terms = pd.DataFrame(data, columns=['Name', 'IDF']).sort_values(['IDF', 'Name'])

# The following cell prints the 10 terms with the lowest/highest idf, which you can use to check if your results appear correct:

In [13]:
"""Print first 10/last 10 terms."""

print(f"Terms with the lowest idf:\n{terms[:10]}\n")
print(f"Terms with the highest idf:\n{terms[-10:]}")

Terms with the lowest idf:
          Name       IDF
6081      game  1.296180
11929     play  1.416360
5405   feature  1.472359
5828      free  1.584695
10609      new  1.665665
17779    world  1.781792
16050     time  1.809229
689        app  1.838871
5928       fun  1.859132
16913      use  1.873860

Terms with the highest idf:
        Name       IDF
21346  회원가입에  7.693943
21347    회원을  7.693943
21348    획득한  7.693943
21349     효과  7.693943
21350    효과음  7.693943
21352    ﬁnd  7.693943
21353  ﬁnger  7.693943
21354  ﬁnish  7.693943
21355   ﬁrst  7.693943
21356    ﬂye  7.693943


## Problem 5: Keyword extraction

We often want to extract salient keywords from a document. A simple method is to pick the $k$ terms with the highest tf–idf value. Your last task in this lab is to implement this method. More specifically, we ask you to implement a function `keywords` that extracts keywords from a text.

In [22]:
def keywords(text, n=10):
    """
    Arguments:
      text (str): The text from which to extract keywords.
      n (int): The number of keywords to extract. [default: 10]

    Returns:
      A list containing the `n` most salient keywords from `text`, as measured by
      their tf–idf value relative to the collection of app descriptions.
    """
    text = vectorizer.transform([text])
    idx = text.nonzero()[1] # indices
    tf_idf = text.data # tf-idf-values
    feature_names = vectorizer.get_feature_names_out()

    # Extract text terms from feature names using idx
    name_vec = [feature_names[i] for i in idx]
    
    data = [(idx, name, tf_idf) for idx, name, tf_idf in zip(idx, name_vec, tf_idf)]
    
    return pd.DataFrame(data, columns=['Index', 'Name', 'TF-IDF']).sort_values('TF-IDF', ascending=False)[0:n]['Name']

### 🤞 Test your code

Test your implementation by running the following cell:

In [23]:
"""Check that the most salient keywords from the description of 'Train Conductor World'
   overlap substantially with the expected list of keywords."""

out = keywords(df['description'][1428])
print(out)
assert len(out) == 10
assert len(
    set(out) & set(['train', 'railway', 'railroad', 'rail', 'chaos', 'crash', 'timetable', 'overcast', 'haul', 'tram'])
) >= 6, "Keywords for df['description'][1428] do not overlap substantially with the expected result"
success()

{'dungeon': 4485, 'crawl': 3347, 'puzzle': 12522, 'rpg': 13415, 'match': 9637, 'game': 6081, 'run': 13441, 'tile': 16021, 'long': 9175, 'good': 6437, 'make': 9470, 'want': 17452, 'cry': 3440, 'toucharcade': 16193, 'monster': 10160, 'block': 1730, 'way': 17515, 'sword': 15435, 'stave': 14945, 'pixel': 11874, 'art': 832, 'orcs': 11172, 'dragon': 4373, 'elemental': 4723, 'dig': 4054, 'backpack': 1179, 'item': 8041, 'cast': 2414, 'magic': 9429, 'spell': 14739, 'change': 2521, 'board': 1783, 'eat': 4586, 'food': 5705, 'energy': 4846, 'boost': 1849, 'battle': 1346, 'get': 6259, 'tough': 16201, 'use': 16913, 'loot': 9190, 'find': 5512, 'blacksmith': 1695, 'armorer': 815, 'upgrade': 16858, 'gear': 6161, 'epic': 4943, 'status': 14944, 'freedom': 5831, 'cost': 3269, 'million': 9961, 'half': 6738, 'hope': 7189, 'kotaku': 8640, 'fight': 5484, 'give': 6327, 'experience': 5171, 'spend': 14746, 'new': 10609, 'skill': 14320, 'trainer': 16245, 'castle': 2416, 'stone': 15020, 'wood': 17750, 'let': 8943,

##### The cell above prints the most salient keywords from the description of the app "Train Conductor World". The exact output may differ slightly depending on the strategy used to break ties, so the cell only checks if there is a sufficient overlap.

**Congratulations on finishing this lab! 👍**

<div class="alert alert-info">
    
➡️ Don't forget to **test that everything runs as expected** before you submit!

</div>